# Composite AutoML

In [1]:
import pandas as pd

df = pd.read_csv('tmp/selected_data_final.csv')
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

X = df.iloc[:,:-1]
y = df.iloc[:,-1]

df.head()

,"Flow rate__fft_coefficient__coeff_1__attr_""angle""",Flow rate__length,"Zone10_Pressure__fft_coefficient__coeff_2__attr_""real""","Zone11_Pressure__change_quantiles__f_agg_""mean""__isabs_False__qh_1.0__ql_0.4","Zone11_Pressure__fft_aggregated__aggtype_""centroid""","Zone15_Pressure__augmented_dickey_fuller__attr_""teststat""","Zone15_Pressure__change_quantiles__f_agg_""mean""__isabs_True__qh_0.8__ql_0.4",Zone15_Pressure__count_below_mean,"Zone15_Pressure__fft_aggregated__aggtype_""centroid""","Zone15_Pressure__fft_aggregated__aggtype_""kurtosis""",...,"Zone31_Pressure__fft_coefficient__coeff_2__attr_""real""","Zone3_Pressure__fft_coefficient__coeff_2__attr_""angle""","Zone4_Pressure__fft_coefficient__coeff_2__attr_""angle""","Zone5_Pressure__fft_coefficient__coeff_2__attr_""angle""","Zone6_Pressure__change_quantiles__f_agg_""mean""__isabs_False__qh_1.0__ql_0.6","Zone6_Pressure__fft_coefficient__coeff_2__attr_""angle""",Zone6_Pressure__partial_autocorrelation__lag_2,"Zone7_Pressure__augmented_dickey_fuller__attr_""teststat""","Zone7_Pressure__fft_coefficient__coeff_2__attr_""angle""",target
0,-133.156136,949.0,-2.964372e+04,95.686572,59.248009,0.505982,45.611017,568.0,75.028473,5.776310,...,-9.310171e+05,148.489858,-156.469317,-139.481181,102.743651,-170.198348,-0.110227,1.712034,92.717040,0
1,-156.043635,863.0,-1.146507e+06,22.493810,41.514168,-1.069010,113.872216,566.0,50.880973,7.862577,...,-1.793086e+06,126.229099,129.473073,131.994737,22.769477,130.317641,-0.399107,-1.096303,98.660564,1
2,-129.957522,992.0,3.226080e+05,122.991554,56.670234,1.841951,75.079191,648.0,78.834497,5.784401,...,1.614355e+06,80.454668,61.397743,53.669202,127.463038,87.747322,-0.190496,1.056589,78.150238,0
3,-159.279799,867.0,-1.819657e+06,2.819112,37.574585,-1.185652,97.722058,539.0,43.575737,9.189750,...,-8.465515e+05,134.725121,141.749812,143.606770,5.531214,136.339948,-0.368715,-1.208694,99.725616,1
4,-133.498270,991.0,-1.776349e+05,98.944519,58.988249,-0.474574,82.765689,617.0,64.398653,6.897727,...,2.690433e+05,139.196540,130.846438,129.830750,72.038636,133.019809,-0.191307,-0.531102,110.545236,1


In [2]:
# code
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

seed = 0
folds = 10
algorithm = RandomForestClassifier(n_estimators=100, random_state=seed)

def get_score(X, y):    
    # Cross Validation
    score = cross_val_score(algorithm, X, y, cv=folds).mean()
    print("Cross Validation Score: "+ str(score*100) + "%") 
    # Train-Test
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=seed)
    algorithm.fit(X_train, y_train)
    y_pred = algorithm.predict(X_test)
    print(classification_report(y_test, y_pred))

get_score(X, y)

Cross Validation Score: 77.48169562971958%
              precision    recall  f1-score   support

           0       0.78      0.73      0.76      1670
           1       0.74      0.79      0.77      1635

    accuracy                           0.76      3305
   macro avg       0.76      0.76      0.76      3305
weighted avg       0.76      0.76      0.76      3305



### TPOT

In [ ]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, test_size=0.3, random_state=seed)

tpot = TPOTClassifier(generations=50, population_size=50, n_jobs=-1, verbosity=2)
tpot.fit(X_train, y_train)

Generation 1 - Current best internal CV score: 0.773293735970006
Generation 2 - Current best internal CV score: 0.7738642478471992
Generation 3 - Current best internal CV score: 0.7738642478471992
Generation 4 - Current best internal CV score: 0.7738642478471992
Generation 5 - Current best internal CV score: 0.7744336408378557
Generation 6 - Current best internal CV score: 0.7767173163457719
Generation 7 - Current best internal CV score: 0.7779987548085405
Generation 8 - Current best internal CV score: 0.7779987548085405
Generation 9 - Current best internal CV score: 0.780854366257451
Generation 10 - Current best internal CV score: 0.780854366257451
Generation 11 - Current best internal CV score: 0.780854366257451
Generation 12 - Current best internal CV score: 0.780854366257451
Generation 13 - Current best internal CV score: 0.780854366257451
Generation 14 - Current best internal CV score: 0.780854366257451
Generation 15 - Current best internal CV score: 0.781284261087567
Generation 1

In [ ]:
tpot.score(X_test, y_test)

In [ ]:
tpot.export('tmp/tpot_composite_pipeline.py')